In [ ]:
%matplotlib inline
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys
import warnings

from itertools import product
from more_itertools import chunked
from functools import partial
from collections import namedtuple
import multifidelityfunctions as mff
import multiLevelCoSurrogates as mlcs

from pyprojroot import here

np.random.seed(20160501)  # Setting seed for reproducibility

module_path = Path().joinpath('..')
if module_path not in sys.path:
    sys.path.append(module_path)

from IPython.core.display import clear_output
from pprint import pprint
np.set_printoptions(linewidth=200, edgeitems=10, precision=4, suppress=True)
plot_dir = here('plots/')
data_dir = here('files/2019-08_MSE_npy/')

In [ ]:
forr_1d = np.load(data_dir.joinpath('Matern_1d_forrester_lin_mse_tracking.npy'))
forr_2d = np.load(data_dir.joinpath('Matern_2d_forrester_lin_mse_tracking.npy'))

In [ ]:
forr_1d.shape

In [ ]:
num_colors = 5

norm1 = colors.LogNorm(vmin=1e-3, vmax=1e2, clip=True)
norm2 = colors.LogNorm(vmin=1e-1, vmax=2.5e1, clip=True)
discrete_cmap = plt.get_cmap('viridis_r', num_colors)

In [ ]:
plt.imshow(forr_1d[:,:,0,1], norm=norm1, origin='lower', cmap=discrete_cmap)
plt.colorbar()
plt.show()
plt.imshow(forr_1d[:,:,0,0], norm=norm1, origin='lower',  cmap=discrete_cmap,)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(np.median(forr_1d[:,:,:,1], axis=2), norm=norm1, origin='lower', cmap=discrete_cmap)
plt.colorbar()
plt.axhline(10, color='black', alpha=.5)
plt.axhline(20, color='black', alpha=.5)
plt.axhline(30, color='black', alpha=.5)
plt.axhline(40, color='black', alpha=.5)
plt.show()
plt.imshow(np.median(forr_1d[:,:,:,0], axis=2), norm=norm1, origin='lower',  cmap=discrete_cmap,)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(np.mean(forr_2d[:,:,:,1], axis=2), norm=norm2, origin='lower', cmap=discrete_cmap)
plt.colorbar()
plt.axhline(10, color='black', alpha=.5)
plt.axhline(20, color='black', alpha=.5)
plt.axhline(30, color='black', alpha=.5)
plt.axhline(40, color='black', alpha=.5)
plt.show()
plt.imshow(np.mean(forr_2d[:,:,:,0], axis=2), norm=norm2, origin='lower',  cmap=discrete_cmap,)
plt.colorbar()
plt.show()

In [ ]:
from scipy.spatial.distance import pdist
from pyDOE import lhs

In [ ]:
def low_lhs_sample(ndim, nlow):
    if ndim == 1:
        return np.linspace(0,1,nlow).reshape(-1,1)
    elif ndim > 1:
        return lhs(ndim, nlow)

In [ ]:
save_name = data_dir.joinpath('minimum_distance_sampling.npy')
dims = [1,2,4,6,8]

if not save_name.exists():
    dists = np.empty((51,126,50,10))
    dists[:] = np.nan

    for i,j,k,ndim in product(range(51), range(126), range(50), dims):
        if i < 2 or not j > i:
            continue
        sample = low_lhs_sample(ndim, j)
        subsample = sample[np.random.choice(j, i, replace=False)]
        dists[i,j,k,ndim] = np.min(pdist(subsample))

    np.save(save_name, dists)
else:
    dists = np.load(save_name)

In [ ]:
!ls -lh /tank/rijnsjvan/mlcs-files/2019-08_MSE_npy | grep minimum

In [ ]:
num_colors=9
discrete_cmap = plt.get_cmap('viridis', num_colors)

norms = [colors.LogNorm(vmin=1e-2, vmax=1, clip=True),
         colors.LogNorm(vmin=1e-2, vmax=1, clip=True),
         colors.LogNorm(vmin=1e-1, vmax=1, clip=True),
         colors.LogNorm(vmin=2e-1, vmax=1, clip=True),
         colors.LogNorm(vmin=3e-1, vmax=1, clip=True),]

for norm, ndim in zip(norms, dims):
    
    plt.figure(figsize=(9,3.5))
#     plt.imshow(np.mean(dists[:,:,:,ndim], axis=2), norm=norm, origin='lower', cmap='viridis_r')
    plt.imshow(np.mean(dists[:,:,:,ndim], axis=2), origin='lower', cmap=discrete_cmap, norm=norm)
    plt.xlabel('$j$')
    plt.ylabel('$i$')
    plt.title(f"Mean minimum distance between $i$ points when randomly\n chosen from $j$ evenly spaced points ({ndim}D).")
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(plot_dir.joinpath(f'distance_problem_{ndim}.pdf'))
    plt.show()

In [ ]:
def quicktest():
    max_high = 51
    max_low = 52
    num_iters = 10
    dists = np.empty((max_high,max_low,num_iters,10))
    dists[:] = np.nan

    dims = [1,2,4,6,8]
    # ndim = 1
    for i,j,k,ndim in product(range(max_high), range(max_low), range(num_iters), dims):
        if i < 2 or not j > i:
            continue
        sample = low_lhs_sample(ndim, j)
        subsample = sample[np.random.choice(j, i, replace=False)]
        dists[i,j,k,ndim] = np.min(pdist(subsample))


In [ ]:
%prun quicktest()

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f quicktest quicktest()